In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from IPython.display import clear_output

In [18]:
data = pd.read_csv('data_4.csv')

data = data.drop(['REF_DATE'], axis=1)

y = data.YoY_change_1
X = data.drop(columns=['YoY_change_1'])

categorical_features = ['GEO', 'Category', 'Farm products', 'CattleType']

label_encoders = {}
for feature in categorical_features:
    le = LabelEncoder()
    X[feature] = le.fit_transform(X[feature])
    label_encoders[feature] = le

print(X.shape, y.shape)

n_test = int(X.shape[0] * .2)
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y, test_size = n_test, shuffle=False)

print(Xtrain.shape,Xtest.shape)

(65298, 28) (65298,)
(52239, 28) (13059, 28)


In [35]:
rf_regressor = RandomForestRegressor(random_state=42, n_estimators=1000, min_samples_leaf=2, min_samples_split=2, max_features='sqrt')
rf_regressor.fit(Xtrain, ytrain)

RandomForestRegressor(max_features='sqrt', min_samples_leaf=2,
                      n_estimators=1000, random_state=42)

In [ ]:
# Set up the cross-validation scheme
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10],
    'n_estimators': [25, 50, 100, 200],
    'max_depth': [2, 5, 10, 20]
    }

# Initialize the Random Forest Regressor
rf_regressor = RandomForestRegressor(random_state=42)

# Initialize GridSearchCV with cross-validation
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, cv=kf, scoring='neg_mean_squared_error')

# Fit GridSearchCV to find the best parameter
grid_search.fit(Xtrain, ytrain)

In [ ]:
# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best Parameters: {best_params}')
print(f'Best Cross-Validation Score: {best_score:.4f}')

# Train the Random Forest Regressor with the best parameters
best_rf_regressor = RandomForestRegressor(**best_params, random_state=42)
best_rf_regressor.fit(Xtrain, ytrain)